#### DynamoDB Integration with API Gateway
* We will use API Gateway endpoints, lambda functions and S3 Events to create and query our COVID-19 database on DynamoDB

#### Insert Records into DynamoDB table using API Gateway Triggers
* Create DynamoDB table.
    * Navigate to AWS DynamoDB Console and Click on Create table. Give a name to your table (I have given the name covid), Give a name and data type for Partition key (In my case id with data type String) and Click Create table.
    <img src='Create_Table.png' width=700>
    * Create an IAM Role to Allow Put Item Access for API Gateway Service to Dynamo DB.
        * Under Access Management in IAM Service Go to Roles and Click on Create Role.
        * Choose API Gateway service, give a name to your Role and Click Create Role.
        <img src='Create_Role.png' width=700>
        * Go to this Role and Click on Attach Policies and choose the AmazonDynamoDBFullAccess Policy and Click on Attach Policy.
        <img src='Attach_Permissions.png' width=700>
    * Create an API Endpoint and Create POST Method.
        * Go to API Gateway Service. Under REST API choose Build. Give name to your API and Click Create API.
        <img src='Create_API.png' width=700>
        * Under Resources > Actions Select Create Resource. Give a Resource name and Click Create Resource.
        <img src='Create_Resource.png' width=700>
        * Select the putdynamodbdata resource and Under Actions choose Create Method and Choose POST Method.
        * Under Integration Type choose AWS Service. Select the service as DynamoDB. Action would be PutItem. Attach your newly created IAM Role and Click Save.
        <img src='Integration_Type.png' width=700>
    * Configure the POST Method
        * Under Integration Request, Choose Mapping Templates. Click Add mapping template. Under Content-Type write application/json.
        * Mapping templates allow us to enable us to define the Request Structure API will use and then transform the request into the structure that the DynamoDB API (PutItem) requires.
        <img src='Mapping_Template.png' width=700>
        * Template for sending request Item as parameters dynamically.
            ```"token": {"S": "$input.params('tokenvalue')"}```            
        * Go to Actions > Deploy API.
        * Copy this API Deploy Stage URL into Postman for API Testing.
            * Under Headers Set Content-Type as application/json.
            * Under Body paste the first record to be inserted into DynamoDB table in JSON format.
            <code>
            {"prname": "Ontario",
            "date": "04-12-2020",
            "numconf": "123526",
            "numdeaths": "3737",
            "numtested": "6251327",
            "numrecover": "104792"}
            </code>
            * Click on Send. Status code 200 is returned.
            <img src='Postman.png' width=700>
            * A new record is inserted into your DynamoDB Table.
            <img src='Item_Inserted.png' width=700>

#### Bulk Update DynamoDB records using Amazon S3 file update as trigger
* Under Lambda Service, Create a new Lambda function with Python 3.8 as runtime.
* lambda_function configuration:
    * Using boto3 get the client for S3.
    * Read the file uploaded to s3 using get_object function.
    * The information about file uploaded to s3 is part of the lambda event object.
    * Call the dynamodb resource using the boto3 object.
    * Create resource mapping for each item in the dynamodb table for put_item API.
    <code>
    import boto3
    s3_client = boto3.client("s3")
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('covidCA')
    def lambda_handler(event, context):
        bucket_name = event['Records'][0]['s3']['bucket']['name']
        s3_file_name = event['Records'][0]['s3']['object']['key']
        resp = s3_client.get_object(Bucket=bucket_name, Key=s3_file_name)
        data = resp['Body'].read().decode("utf-8")
        records = data.split("\n")
        for rec in records:
            rec_item = rec.split(",")
            #Insert into dynamodb
            table.put_item(
                Item = {
                    "id": rec_item[0],
                    "prname": rec_item[1],
                    "prname": rec_item[2],
                    "numconf": rec_item[3],
                    "numdeaths": rec_item[4],
                    "numtested": rec_item[5],
                    "numrecover": rec_item[6],
                    "numtoday": rec_item[7],
                    "numdeathstoday": rec_item[8],
                    "numtestedtoday": rec_item[9],
                    "numrecoveredtoday": rec_item[10]
                }
                )
    </code>
    <img src='Lambda_Function.png' width=700>
    * S3 bucket configuration.
        * Upload your csv file to S3 bucket.
        * Under your S3 bucket properties, Under Event notifications Click on Create event notification.
        <img src='Event_Notification.png' width=700>
        * Choose suffix as .csv and Event types as All object create events.
        <img src='Event_Type.png' width=700>
        * Under Destination choose Lambda function and select your Lambda function.
        <img src='Destination.png' width=700>
        * Now uploading a file into the buckets leads to creation of items in DynamoDB table.
        <img src='Upload_To_S3.png' width=700>

#### Query DynamoDB table using API Gateway
* Configure the GET method.
    * Under your API Endpoint Resource putdynamodbdata, create a new GET method.
    * Select your Resource, Under Actions choose Create Method. Select GET method.
    * Under Integration Type select AWS Service and select the AWS Service as DynamoDB.
    * HTTP Method is POST (Since we need to Pass JSON structured data as request to query the data) with Action Query. Paste your APIGatewayToDynamoDBRole ARN and Click Save.
    <img src='Get_Method.png' width=700>
* Configure the mapping template for GET Request.
    * Content-type is application/json.
    * Generate template.
        * Provide the partition key as the KeyConditionExpression.
        * Expression attribute value is passed as part of a query string parameter. We will pass partition key id as part of query string parameter. Deploy the API.
        <img src='Add_Template.png' width=700>
    * Use Postman tool to invoke the API.
        * Paste the Invoke URL in front of the GET Action.
        * Under Headers, enter Content-type as application/json.
        * Pass the query string parameter as ?id=4098 along with the URL. Click Send.
        <img src='Query_String.png' width=700>
        * The request returns the following response
        <code>
        {
        "Count": 1,
        "Items": [
            {
                "prname": {
                    "S": "Quebec"
                },
                    "date": {
                    "S": "05-12-2020"
                },
                "numdeathstoday": {
                    "S": "48"
                },
                "numdeaths": {
                    "S": "7231"
                },
                "numtoday": {
                    "S": "2031"
                },
                "numrecoveredtoday": {
                    "S": "1279"
                },
                "id": {
                    "S": "4098"
                },
                "numconf": {
                    "S": "149908"
                },
                "numrecover": {
                    "S": "128828"
                },
                "numrtestedtoday": {
                    "S": "11322"
                },
                "numtested": {
                    "S": "2238113"
                }
            }
        ],
        "ScannedCount": 1
    }
    </code>